# Lecture 3 (1/2): Constraint Programming
This notebook demonstrates how constraint programming works. We use the `python-constraint` library.

# Team making
Let's say we have 20 students. We want to make groups of 1 or 2 students for their assignment.
Formulate this as a constraint programming problem.

In [1]:
from constraint import *
from itertools import product

In [2]:
num_students = 20
students = list(range(num_students))
partners = list(range(num_students))
print(f"Students: {students}")
print(f"Partner: {partners}")

Students: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Partner: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [3]:
sp = list(product(students, partners))
print(f"Pairs: {sp[:10]}...")

Pairs: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9)]...


Using a constraint solver requires declaration of a problem. We then specify variables that are handled in that probelm.

In [4]:
problem = Problem()
problem.addVariables(sp, [0, 1])

In [5]:
# A student can pick only one partner
for s in students:
    pairs = [(s, p) for p in partners]
    problem.addConstraint(ExactSumConstraint(1), pairs)

Let's see how to obtain a solution so far. You can either get one solution using `problem.getSolution()` or all the solutions using `problem.getSolutions()` (or `problem.getSolutionIter()` which returns an iterator).

In [6]:
# Get one solution
sol = problem.getSolution()
print(list(filter(lambda item: item[1] == 1, sol.items())))

[((0, 0), 1), ((1, 0), 1), ((2, 0), 1), ((3, 0), 1), ((4, 0), 1), ((5, 0), 1), ((6, 0), 1), ((7, 0), 1), ((8, 0), 1), ((9, 0), 1), ((10, 0), 1), ((11, 0), 1), ((12, 0), 1), ((13, 0), 1), ((14, 0), 1), ((15, 0), 1), ((16, 0), 1), ((17, 0), 1), ((18, 0), 1), ((19, 0), 1)]


Take a look at the first tuple, `((0,0), 1)`. The first element `(0, 0)` is a tuple of student ids. The second element `1` indicates the value of the variable. Together, it indicates that $X_{0, 0} = 1$. That is, the student 0 is partnering up with himself/herself.

See the second tuple, `(1, 0), 1`. This indicates that the student 1 is partnering up with student 0. But as we have seen, student 0 is already pairing up with themselves. So such solution should be prohibited. Let's add another constraint to prevent this value to be a solution.

In [7]:
# Partners should be symmetric
for s in students:
    for p in partners:
        if s < p:
            pairs = [(s, p), (p, s)]
            problem.addConstraint(AllEqualConstraint(), pairs)

Here, I am adding a constraint to force that $X_{i,j}$ is equal to the value of $X_{j, i}$.

In [8]:
# Get some solutions
solutions = problem.getSolutionIter()

for i, sol in enumerate(solutions):
    print(list(filter(lambda item: item[1] == 1, sol.items())))
    if i >= 5:
        break

[((0, 1), 1), ((1, 0), 1), ((2, 3), 1), ((3, 2), 1), ((4, 5), 1), ((5, 4), 1), ((6, 7), 1), ((7, 6), 1), ((8, 9), 1), ((9, 8), 1), ((10, 11), 1), ((11, 10), 1), ((12, 13), 1), ((13, 12), 1), ((14, 15), 1), ((15, 14), 1), ((16, 17), 1), ((17, 16), 1), ((18, 19), 1), ((19, 18), 1)]
[((0, 1), 1), ((1, 0), 1), ((2, 3), 1), ((3, 2), 1), ((4, 5), 1), ((5, 4), 1), ((6, 7), 1), ((7, 6), 1), ((8, 9), 1), ((9, 8), 1), ((10, 11), 1), ((11, 10), 1), ((12, 13), 1), ((13, 12), 1), ((14, 15), 1), ((15, 14), 1), ((16, 17), 1), ((17, 16), 1), ((18, 18), 1), ((19, 19), 1)]
[((0, 1), 1), ((1, 0), 1), ((2, 3), 1), ((3, 2), 1), ((4, 5), 1), ((5, 4), 1), ((6, 7), 1), ((7, 6), 1), ((8, 9), 1), ((9, 8), 1), ((10, 11), 1), ((11, 10), 1), ((12, 13), 1), ((13, 12), 1), ((14, 15), 1), ((15, 14), 1), ((16, 18), 1), ((18, 16), 1), ((17, 19), 1), ((19, 17), 1)]
[((0, 1), 1), ((1, 0), 1), ((2, 3), 1), ((3, 2), 1), ((4, 5), 1), ((5, 4), 1), ((6, 7), 1), ((7, 6), 1), ((8, 9), 1), ((9, 8), 1), ((10, 11), 1), ((11, 10), 

Let's say some students have already formed pairs. We can create a list of such pairs $(i, j)$ and add constraints to enforce $X_{i, j}=1$.

In [9]:
# Some students have formed a pair already
fixe_pairs = [(2, 3), (5, 6), (7, 9), (10, 12), (14, 17), (18, 19)]

problem.addConstraint(ExactSumConstraint(len(fixe_pairs)), fixe_pairs)

In [10]:
# Get all feasible solutions
solutions = problem.getSolutions()
print(f"The size of feasible solutions is: {len(solutions)}")

for sol in solutions:
    sol = list(filter(lambda item: item[1] == 1, sol.items()))
    print(sol)

The size of feasible solutions is: 764
[((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((0, 1), 1), ((1, 0), 1), ((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 16), 1), ((16, 15), 1)]
[((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((0, 1), 1), ((1, 0), 1), ((4, 8), 1), ((8, 4), 1), ((11, 13), 1), ((13, 11), 1), ((15, 15), 1), ((16, 16), 1)]
[((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7), 1), ((12, 10), 1), ((17, 14), 1), ((19, 18), 1), ((0, 1), 1), ((1, 0), 1), ((4, 8), 1), ((8, 4), 1), ((11, 15), 1), ((15, 11), 1), ((13, 16), 1), ((16, 13), 1)]
[((2, 3), 1), ((5, 6), 1), ((7, 9), 1), ((10, 12), 1), ((14, 17), 1), ((18, 19), 1), ((3, 2), 1), ((6, 5), 1), ((9, 7)

In [11]:
problem.reset()

# Design Exploration

In [12]:
problem = Problem()

font_sizes = [12, 16, 20, 24, 28, 32]
color_range = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


problem.addVariable('h1-red', color_range)
problem.addVariable('h1-green', color_range)
problem.addVariable('h1-blue', color_range)
problem.addVariable('h1-size', font_sizes)

problem.addVariable('h2-red', color_range)
problem.addVariable('h2-green', color_range)
problem.addVariable('h2-blue', color_range)
problem.addVariable('h2-size', font_sizes)

problem.addVariable('p-red', color_range)
problem.addVariable('p-green', color_range)
problem.addVariable('p-blue', color_range)
problem.addVariable('p-size', font_sizes)

In [13]:
problem.getSolution()

{'h1-size': 32,
 'h2-size': 32,
 'p-size': 32,
 'h1-blue': 100,
 'h1-green': 100,
 'h1-red': 100,
 'h2-blue': 100,
 'h2-green': 100,
 'h2-red': 100,
 'p-blue': 100,
 'p-green': 100,
 'p-red': 100}

In [14]:
accepted_colors = [(30, 30, 30), (60, 60, 60)]

def color_constraint(r, g, b):
    if (r, g, b) in accepted_colors:
        return True
    else:
        return False

problem.addConstraint(
    color_constraint,
    ['h1-red', 'h1-green', 'h1-blue']
)

problem.addConstraint(
    color_constraint,
    ['h2-red', 'h2-green', 'h2-blue']
)

problem.addConstraint(
    color_constraint,
    ['p-red', 'p-green', 'p-blue']
)

In [15]:
print(f"{len(problem.getSolutions())}")
sol_iter = problem.getSolutionIter()
for i, sol in enumerate(sol_iter):
    print(sol)
    if i >= 5:
        break

1728
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-green': 60, 'p-red': 60, 'h1-size': 32, 'h2-size': 32, 'p-size': 32}
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-green': 60, 'p-red': 60, 'h1-size': 32, 'h2-size': 32, 'p-size': 28}
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-green': 60, 'p-red': 60, 'h1-size': 32, 'h2-size': 32, 'p-size': 24}
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-green': 60, 'p-red': 60, 'h1-size': 32, 'h2-size': 32, 'p-size': 20}
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-green': 60, 'p-red': 60, 'h1-size': 32, 'h2-size': 32, 'p-size': 16}
{'h1-blue': 60, 'h1-green': 60, 'h1-red': 60, 'h2-blue': 60, 'h2-green': 60, 'h2-red': 60, 'p-blue': 60, 'p-gr

In [16]:
def font_size_constraint(size1, size2):
    return size1 > size2

problem.addConstraint(font_size_constraint, ['h1-size', 'h2-size'])
problem.addConstraint(font_size_constraint, ['h2-size', 'p-size'])

In [17]:
print(f"{len(problem.getSolution())}")

12


In [18]:
problem.getSolution()

{'h2-size': 28,
 'h1-size': 32,
 'p-size': 24,
 'h1-blue': 60,
 'h1-green': 60,
 'h1-red': 60,
 'h2-blue': 60,
 'h2-green': 60,
 'h2-red': 60,
 'p-blue': 60,
 'p-green': 60,
 'p-red': 60}

In [19]:
problem.reset()